In [1]:
import compas.geometry as cg
from arch import construct_arch
from problem.variables import Problem
from problem.helpers import print_matrix
import numpy as np
import cvxpy as cp
from compas_viewer import Viewer
from compas.colors import Color
import copy
from compas_viewer.scene.tagobject import Tag


In [2]:
n = 40
Arch_Object = construct_arch(5.0, 20.0, n, 1.0)
Arch_Object_0 = copy.deepcopy(Arch_Object)
The_Problem = Problem.from_polygons(Arch_Object)

Setting BC for Block ID: 0
Setting BC for Block ID: 39


In [3]:
c = The_Problem.C # to avoid singularities
#c[6] = 1.0  # small force to avoid singularity
print("The Problem C vector:\n", c)
u = The_Problem.U
intfn = The_Problem.intfn
A_ub = The_Problem.A_ub
A_eq = The_Problem.A_eq

b_ub = np.zeros(A_ub.shape[0])
b_eq = np.zeros(A_eq.shape[0])

print(f"number of interfaces: {intfn}")
def set_BCs(b, Idx, vector):
    """Sets the boundary conditions in the RHS vector b"""
    b[Idx*2    ] = vector[0]
    b[Idx*2 + 1] = vector[1]
    return b

def set_force_BCs(c, Idx, vector):
    """Set force boundary conditions in the global force vector C"""
    c[Idx*3    ] = vector[0]
    c[Idx*3 + 1] = vector[1]
    c[Idx*3 + 2] = vector[2]
    return c
for i in range(n):
    set_force_BCs(c, i, (0.0, -9.81, 0.0))

index_BC_x = [3,5]
for i in index_BC_x:
    set_force_BCs(c, i, (-1.0, 0.0, -5))

#set_force_BCs(c, 2, (0.0, 0.0, 1.0))
#set_force_BCs(c, n-2, (0.0, 0.0, 1.0))

# set_BCs(b_ub, 0, (0.01, 0.001))
# set_BCs(b_eq, 0, (-0.001, -0.001))
# set_BCs(b_ub, intfn-3, (0.01, 0))
# set_BCs(b_eq, intfn-3, (0.0, 0))


The Problem C vector:
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
number of interfaces: 39


In [4]:
U = cp.Variable(3*n)

constraints = [
    A_ub @ U >= b_ub,
    A_eq @ U == b_eq,
    U[0:3] == 0.0,
    U[(n-1)*3 : n*3] == 0.0,
    U[7] == 0.1,
]

objective = cp.Minimize(-(c @ U))  # minimize −c·U  (equiv. maximize c·U)
prob = cp.Problem(objective, constraints)

prob.solve(solver=cp.MOSEK, verbose=False)  # or CBC / ECOS

print("Feasible:", prob.status)
U_star = U.value
print(U_star)

Feasible: optimal
[-0.         -0.         -0.         -0.00792174  0.05113244 -0.08773771
 -0.01420448  0.1        -0.00573213 -0.01175547  0.10243532 -0.00573213
 -0.00942196  0.10498153 -0.00573213 -0.00720898  0.10763316 -0.00573213
 -0.01195841  0.1110746   0.00616159 -0.01406305  0.10801629  0.00616159
 -0.01602368  0.10486371  0.00616159 -0.01783609  0.10162366  0.00616159
 -0.01949638  0.09830308  0.00616159 -0.02100098  0.09490912  0.00616159
 -0.02234666  0.09144907  0.00616159 -0.02353053  0.08793037  0.00616159
 -0.02455004  0.08436058  0.00616159 -0.02540301  0.08074737  0.00616159
 -0.02608759  0.07709852  0.00616159 -0.02660231  0.07342185  0.00616159
 -0.02694608  0.06972528  0.00616159 -0.02711815  0.06601675  0.00616159
 -0.02711815  0.06230423  0.00616159 -0.02694608  0.05859571  0.00616159
 -0.02660231  0.05489914  0.00616159 -0.02608759  0.05122247  0.00616159
 -0.02540301  0.04757362  0.00616159 -0.02455004  0.04396041  0.00616159
 -0.02353053  0.04039062  0.00616

In [5]:
U_new = U_star.reshape((n, 3))
print_matrix(A_ub)
for i,polygon in enumerate(Arch_Object):
    dx, dy, rz = U_new[i]
    polygon.translate(cg.Vector(dx, 0,dy))
    polygon.rotate(rz, axis=cg.Vector(0,1, 0), point=polygon.centroid())



<IPython.core.display.Math object>

In [6]:
viewer = Viewer()
viewer.scene.clear()
for idx, polygon in enumerate(Arch_Object):
    viewer.scene.add(polygon,facecolor=Color(0.0, 0.0, 1.0), show_points=True, pointcolor=Color(1.0, 0.0, 1.0), linecolor=Color(1.0, 0.5, 1.0), linewidth=1, opacity=0.5)
    t1 = Tag(text=str(idx), position=polygon.centroid(), color=Color(1.0, 0.0, 0.0))
    viewer.scene.add(t1)
# for polygon in Arch_Object_0:
#     viewer.scene.add(polygon,facecolor=Color(0.0, 1.0, 0.0),opacity=0.15)

viewer.show()